# Faster Video in OpenCV
**The fast is slow and the slow is fast .... read the comments on the blog.**

I have you found that working with video file via OpenCV’s `cv2.VideoCapture` can be slow and sluggish.
It seem like the `cv2.VideoCapture` and the associated `.read` method to poll another frame from your video file
is taking far to long.
The problem is likely the video compression and frame decoding.
Depending on your video file type, the codecs you have installed, and the physical hardware of your machine,
much of your video processing pipeline can actually be consumed by reading and decoding
the next frame in the video file.

The problem is that you’re both reading and decoding the frame in your main processing thread!
The `.read`  method is a [blocking operation][01].
That is, the main thread of your Python + OpenCV application is entirely blocked
(i.e., stalled) until the frame is read from the video file, decoded, and returned to the calling function.
By moving these blocking I/O operations to a separate thread and maintaining a queue of decoded frames,
your can increase the frame processing rate.

The code below show how this latency can be imporved by moving
the reading and decoding of video file frames to an entirely separate thread of the program,
freeing up our main thread to handle the actual image processing.
[01]:https://en.wikipedia.org/wiki/Blocking_(computing)

## The Slow, Naive Method
This is the slower, **not recommend** method for processing the video.
The goal of this section is to obtain a baseline on our video frame processing
throughput rate using OpenCV and Python.

In [1]:
# import the necessary packages
from imutils.video import FPS
import numpy as np
import imutils
import cv2

# open a pointer to the video stream and start the FPS timer
print("[INFO] starting video file...")
stream = cv2.VideoCapture("videos/Jurassic-Park-Trailer-1993.mp4")
fps = FPS().start()

# loop over frames from the video file stream
while True:
    # grab the frame from the threaded video file stream
    (grabbed, frame) = stream.read()

    # if the frame was not grabbed, then we have reached the end of the stream
    if not grabbed:
        break

    # resize the frame and convert it to grayscale (while still retaining 3 channels)
    frame = imutils.resize(frame, width=450)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame = np.dstack([frame, frame, frame])

    # display a piece of text to the frame (so we can benchmark
    # fairly against the fast method)
    cv2.putText(frame, "Slow Method", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)    #noqa

    # show the frame and update the FPS counter
    cv2.imshow("Frame", frame)
    cv2.waitKey(1)
    fps.update()

# stop the timer and display FPS information
fps.stop()
print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))

# do a bit of cleanup
stream.release()
cv2.destroyAllWindows()

[INFO] starting video file...
[INFO] elasped time: 7.53
[INFO] approx. FPS: 126.97


* `cv2.waitKey(1)` gives you approxamately 127 FPS and time of 7.5 seconds
* `cv2.waitKey(25)` gives you approxamately 32 FPS and time of 30 seconds

## The Faster, Threading Method

In [10]:
# import the necessary packages
from imutils.video import FileVideoStream
from imutils.video import FPS
import numpy as np
import imutils
import time
import cv2

# start the file video stream thread and allow the buffer to start to fill
print("[INFO] starting video file thread...")
fvs = FileVideoStream("videos/Jurassic-Park-Trailer-1993.mp4").start()
time.sleep(1.0)

# start the FPS timer
fps = FPS().start()

# loop over frames from the video file stream
while fvs.more():
    # grab the frame from the threaded video file stream, resize
    # it, and convert it to grayscale (while still retaining 3 channels)
    frame = fvs.read()
    frame = imutils.resize(frame, width=450)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame = np.dstack([frame, frame, frame])

    # display the size of the queue on the frame
    cv2.putText(frame, "Fast Method - Queue Size: {}".format(fvs.Q.qsize()), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)  #noqa

    # show the frame and update the FPS counter
    cv2.imshow("Frame", frame)
    cv2.waitKey(1)
    fps.update()

# stop the timer and display FPS information
fps.stop()
print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))

# do a bit of cleanup
cv2.destroyAllWindows()
fvs.stop()

[INFO] starting video file thread...
[INFO] elasped time: 44.17
[INFO] approx. FPS: 21.64


The `Jurassic-Park-Trailer-1993.mp4` video is 31 seconds long.
But the processing of all the individual frames of the video clip takes approximately 45 seconds,
and with a frame per second (FPS) processing rate of approximately 21.

These results imply that it’s actually taking longer to read and decode the individual frames than the actual length of the video clip!

* `cv2.waitKey(1)` gives you approxamately 22 FPS and time of 44 seconds

## Sources
* [Faster video file FPS with cv2.VideoCapture and OpenCV](http://www.pyimagesearch.com/2017/02/06/faster-video-file-fps-with-cv2-videocapture-and-opencv/)